# Wprowadzenie do PySpark SQL #

źródło: https://sparkbyexamples.com/pyspark/pyspark-sql-with-examples/

Pyspark.sql to moduł w PySpark, który służy do wykonywania operacji podobnych do SQL.

### PySpark SQL DataFrame API ###

DataFrame to rozpartycjonowany zbiór danych zorganizowanych w nazwane kolumny. Koncepcyjnie jest odpowiednikiem tabeli w relacyjnej bazie danych lub ramki danych w R/Python, ale z większą możliwością optymalizacji prędkości zapytań. DataFrames można konstruować z szerokiej gamy źródeł, takich jak pliki z danymi, tabele w Hive, zewnętrzne bazy danych lub istniejące RDD.

PySpark DataFrame jest bardzo podobny do Pandas DataFrame, z wyjątkiem tego, że PySpark DataFrames są podzielone na węzły (co oznacza, że dane w DataFrame są przechowywane na różnych jednostkach w klastrze), a wszelkie operacje w PySpark są wykonywane równolegle na wszystkich partycjach, podczas gdy Panda Dataframe przechowuje dane i oblicza wyniki na jednej jednostce.

### Uruchamianie zapytań SQL w PySpark ###

PySpark SQL jest jednym z najczęściej używanych modułów PySpark. Po stworzeniu ramki można wykonywać na niej operacje przy użyciu składni SQL. Innymi słowy, Spark SQL zapewnia zapytania RAW SQL w Spark, co oznacza, że można korzystać z tradycyjnego ANSI SQL pracując na ramce sparkowej.

Aby użyć zapytania SQL najpierw trzeba stworzyć tabelę tymczasową za pomocą funkcji createOrReplaceTempView(). Po wykonaniu tego kroku tabela będzie dostępna dla całej sesji SparkSession po użyciu funkcji sql(). Zostanie usunięta wraz z zakończeniem sesji.

Wystarczy użyć metody sql() obiektu SparkSession, aby uruchomić zapytanie, a metoda zwróci nową ramkę.

### PySpark SQL przykłady ###

**Tworzenie widoku SQL**




In [3]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
# Może chwilę potrwać
spark = SparkSession.builder.appName("UczymySięSparka").getOrCreate()
spark

df = spark.read.option("header",True) \
          .csv("simple-zipcodes.csv")
df.printSchema()
df.show()

root
 |-- RecordNumber: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- State: string (nullable = true)

+------------+-------+-------------------+-------+-----+
|RecordNumber|Country|               City|Zipcode|State|
+------------+-------+-------------------+-------+-----+
|           1|     US|        PARC PARQUE|    704|   PR|
|           2|     US|PASEO COSTA DEL SUR|    704|   PR|
|          10|     US|       BDA SAN LUIS|    709|   PR|
|       49347|     US|               HOLT|  32564|   FL|
|       49348|     US|          HOMOSASSA|  34487|   FL|
|       61391|     US|  CINGULAR WIRELESS|  76166|   TX|
|       61392|     US|         FORT WORTH|  76177|   TX|
|       61393|     US|           FT WORTH|  76177|   TX|
|       54356|     US|        SPRUCE PINE|  35585|   AL|
|       76511|     US|           ASH HILL|  27007|   NC|
|           4|     US|    URB EUGENE RICE|    704|   PR|
|

Aby korzystać z zapytania ANSI SQL podobnego do zapytań RDBMS, należy utworzyć tymczasową tabelę, odczytując dane z pliku CSV.

In [4]:
spark.read.option("header",True) \
          .csv("simple-zipcodes.csv") \
          .createOrReplaceTempView("Zipcodes") ### to tworzy widok

**PySpark SQL - Select**

select() jest wykorzystywany do wyboru konkretnych kolumn z ramki.


In [5]:
# PySpark: 

df.select("country","city","zipcode","state").show(5)

+-------+-------------------+-------+-----+
|country|               city|zipcode|state|
+-------+-------------------+-------+-----+
|     US|        PARC PARQUE|    704|   PR|
|     US|PASEO COSTA DEL SUR|    704|   PR|
|     US|       BDA SAN LUIS|    709|   PR|
|     US|               HOLT|  32564|   FL|
|     US|          HOMOSASSA|  34487|   FL|
+-------+-------------------+-------+-----+
only showing top 5 rows



SQL:

In [6]:
spark.sql("SELECT country, city, zipcode, state FROM ZIPCODES").show(5)

+-------+-------------------+-------+-----+
|country|               city|zipcode|state|
+-------+-------------------+-------+-----+
|     US|        PARC PARQUE|    704|   PR|
|     US|PASEO COSTA DEL SUR|    704|   PR|
|     US|       BDA SAN LUIS|    709|   PR|
|     US|               HOLT|  32564|   FL|
|     US|          HOMOSASSA|  34487|   FL|
+-------+-------------------+-------+-----+
only showing top 5 rows



**Filtrowanie**

Żeby przefiltrować ramkę można wykorzystać poniższe sposoby:

In [8]:
#PySpark:

df.select("country","city","zipcode","state").where("state == 'AZ'").show(5)

+-------+----+-------+-----+
|country|city|zipcode|state|
+-------+----+-------+-----+
|     US|MESA|  85209|   AZ|
|     US|MESA|  85210|   AZ|
+-------+----+-------+-----+



In [9]:
# SQL where
spark.sql(""" SELECT  country, city, zipcode, state FROM ZIPCODES WHERE state = 'AZ' """).show(5)

+-------+----+-------+-----+
|country|city|zipcode|state|
+-------+----+-------+-----+
|     US|MESA|  85209|   AZ|
|     US|MESA|  85210|   AZ|
+-------+----+-------+-----+



**Sortowanie**

Żeby posortowac wiersze według wartości konretnej kolumny należy wykorzystać orderBy().


In [10]:
df.select("country","city","zipcode","state").where("state in ('PR','AZ','FL')").orderBy("state").show(10)

+-------+-------------------+-------+-----+
|country|               city|zipcode|state|
+-------+-------------------+-------+-----+
|     US|               MESA|  85209|   AZ|
|     US|               MESA|  85210|   AZ|
|     US|               HOLT|  32564|   FL|
|     US|             HOLDER|  34445|   FL|
|     US|          HOMOSASSA|  34487|   FL|
|     US|           HILLIARD|  32046|   FL|
|     US|        PARC PARQUE|    704|   PR|
|     US|PASEO COSTA DEL SUR|    704|   PR|
|     US|       BDA SAN LUIS|    709|   PR|
|     US|    URB EUGENE RICE|    704|   PR|
+-------+-------------------+-------+-----+
only showing top 10 rows



W SQL sortowanie odbywa się za pomocą polecenia ORDER BY.

In [12]:
spark.sql(""" SELECT  country, city, zipcode, state FROM ZIPCODES WHERE state in ('PR','AZ','FL') order by state """).show(10)

+-------+-------------------+-------+-----+
|country|               city|zipcode|state|
+-------+-------------------+-------+-----+
|     US|               MESA|  85209|   AZ|
|     US|               MESA|  85210|   AZ|
|     US|               HOLT|  32564|   FL|
|     US|             HOLDER|  34445|   FL|
|     US|          HOMOSASSA|  34487|   FL|
|     US|           HILLIARD|  32046|   FL|
|     US|        PARC PARQUE|    704|   PR|
|     US|PASEO COSTA DEL SUR|    704|   PR|
|     US|       BDA SAN LUIS|    709|   PR|
|     US|    URB EUGENE RICE|    704|   PR|
+-------+-------------------+-------+-----+
only showing top 10 rows



**Grupowanie**

Grupowanie wykonuje się przy użyciu polecenia groupBy(). Często w parze z poleceniem count()

In [13]:
df.groupBy("state").count().show()

+-----+-----+
|state|count|
+-----+-----+
|   AZ|    2|
|   NC|    3|
|   AL|    3|
|   TX|    3|
|   FL|    4|
|   PR|    5|
+-----+-----+



W SQL tę samą operacją przeprowadza się przy użyciu polecenia GROUP BY.

In [14]:
spark.sql(""" SELECT state, count(*) as count FROM ZIPCODES GROUP BY state""").show()

+-----+-----+
|state|count|
+-----+-----+
|   AZ|    2|
|   NC|    3|
|   AL|    3|
|   TX|    3|
|   FL|    4|
|   PR|    5|
+-----+-----+



### Joiny - PySpark SQL ###

PySpark Join służy do łączenia dwóch lub więcej ramek. PySpark obsługuje wszystkie podstawowe operacje łączenia dostępne w tradycyjnym SQL, takie jak INNER, LEFT OUTER, RIGHT OUTER, LEFT ANTI, LEFT SEMI, CROSS, SELF JOIN. Należy jednak podkreślić, że joiny PySpark to duże transformacje, które obejmują tasowanie danych w obrębie wielu partycji.

PySpark SQL Joins z definicji są bardziej optymalne (dzięki właściwościom ramek sparkowych), jednak nadal istnieją pewne problemy z wydajnością, które należy wziąć pod uwagę podczas używania.

**Składnia joina:**

join(self, other, on=None, how=None)

param other: prawa strona joina
param on: info po czym łączymy
param how: typ joina, argument defaultowy to inner

Do joinów można też dodawać where() i filter(). Można też łączyć po więcej niż jednej kolumnie. 

Nie przerobimy wszystkich typoów joinów, skupimy się na najbardziej podstawowych: inner, left i full outer. W. kolejnym kroku stworzymy ramki do ćwiczeń.

In [15]:

emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()


dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.printSchema()

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+-----

In [18]:
empDF.show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+------+



In [17]:
deptDF.show(truncate=False)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



**Inner Join**

Inner jest domyślnym łączeniem w PySpark i jest najczęściej używanym. To łączy dwa zestawy danych po kolumnach wskazanych jako klucze. Zwróci ramkę, w której znajdą się tylko te wiersze, których klucze sa obecne w obu zbiorach

In [19]:
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"inner").show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



**Full Outer Join**

Zwraca wszystkie wiersze z obu ramek. Tam, gdzie wiersze nie mogą zostać połączone po kluczu zwracany jest null. 

In [20]:
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"outer").show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|null  |null    |null           |null       |null       |null  |null  |Sales    |30     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
|6     |Brown   |2              |2010       |50         |      |-1    |null     |null   |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



**Left Semi Join**

Left semi join w PySpark działa jak tradycyjny left join - zwraca te wiersze, z lewej tabeli, które można połączyć po kluczu z wierszami z prawej tabeli.

In [21]:
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"leftsemi").show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
+------+--------+---------------+-----------+-----------+------+------+

